In [1]:
from qiskit import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_burlington')
from qiskit.ignis.characterization.gates import *
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)

In [2]:
qubits = [0]
controls = [1]

In [ ]:
meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits, qr=5, circlabel='mcal')
cal_job = qiskit.execute(meas_calibs, backend=backend, shots=8192)
cal_results = cal_job.result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')

#Calibration matrix for result C
raw_counts = job.result().get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

# Results with mitigation
mitigated_results = meas_filter.apply(job.result())
mitigated_counts = mitigated_results.get_counts(0)

In [3]:
"""
Taken from https://qiskit.org/documentation/apidoc/characterization.html
1 Qubit Amplitude Cal
"""

circuit, xdata = ampcal_1Q_circuits(75, qubits)
job = qiskit.execute(circuit, backend=backend, shots=8192)
plt.figure(figsize=(10, 6))
initial_theta = 0.02
initial_c = 0.5
initial_phi = 0.1




<Figure size 720x432 with 0 Axes>

In [4]:
job.result().get_counts()

[{'0': 4344, '1': 3848},
 {'0': 4167, '1': 4025},
 {'0': 4336, '1': 3856},
 {'0': 4097, '1': 4095},
 {'0': 4307, '1': 3885},
 {'0': 4123, '1': 4069},
 {'0': 4265, '1': 3927},
 {'0': 4151, '1': 4041},
 {'0': 4258, '1': 3934},
 {'0': 4153, '1': 4039},
 {'0': 4301, '1': 3891},
 {'0': 4234, '1': 3958},
 {'0': 4248, '1': 3944},
 {'0': 4263, '1': 3929},
 {'0': 4255, '1': 3937},
 {'0': 4233, '1': 3959},
 {'0': 4145, '1': 4047},
 {'0': 4319, '1': 3873},
 {'0': 4130, '1': 4062},
 {'0': 4163, '1': 4029},
 {'0': 4188, '1': 4004},
 {'0': 4310, '1': 3882},
 {'0': 4114, '1': 4078},
 {'0': 4325, '1': 3867},
 {'0': 4083, '1': 4109},
 {'0': 4316, '1': 3876},
 {'0': 4028, '1': 4164},
 {'0': 4385, '1': 3807},
 {'0': 4195, '1': 3997},
 {'0': 4328, '1': 3864},
 {'0': 4149, '1': 4043},
 {'0': 4318, '1': 3874},
 {'0': 4164, '1': 4028},
 {'0': 4302, '1': 3890},
 {'0': 4121, '1': 4071},
 {'0': 4405, '1': 3787},
 {'0': 4078, '1': 4114},
 {'0': 4403, '1': 3789},
 {'0': 4010, '1': 4182},
 {'0': 4352, '1': 3840},


In [ ]:
fit = AmpCalFitter(job.result(), xdata, qubits,
                   fit_p0=[initial_theta, initial_c],
                   fit_bounds=([-np.pi, -1],
                               [np.pi, 1]))

# plot the result for the number 1 indexed qubit.
# In this case that refers to Q2 since we passed in as [4, 2])
plt.figure(figsize=(10, 6))
fit.plot(0, ax=plt.gca())

print("Mitigated Rotation Error on U2(0,0): %f rads"%(fit.angle_err()[0]))
plt.show()

In [ ]:
# 1 Qubit Angle Cal
circuit, xdata = anglecal_1Q_circuits(75, qubits)
job = qiskit.execute(circuit, backend=backend, shots=8192)
plt.figure(figsize=(10, 6))
initial_theta = 0.02
initial_c = 0.5
initial_phi = 0.01


fit = AngleCalFitter(job.result(), xdata, qubits,
                   fit_p0=[initial_theta, initial_c],
                   fit_bounds=([-np.pi, -1],
                               [np.pi, 1]))

fit.plot(0, ax=plt.gca())

print("Angle error between X and Y: %f rads"%(fit.angle_err()[0]))
plt.show()

In [ ]:
circs, xdata = ampcal_cx_circuits(150, qubits, controls)
batch1 = circs[:75]
batch2 = circs[75:]
shots = 8192
backend_result1 = qiskit.execute(batch1, backend, shots=shots).result()
backend_result2 = qiskit.execute(batch2, backend, shots=shots).result()



In [ ]:
#concats the results
trunc_parameter = 150

total_result = result.Result.from_dict({'job_id': '5f0769f8c794e4001abdd9ea',
'date': '2020-07-09T19:09:03+00:00',
'backend_version': '2.0.6',
'header': {'backend_name': 'ibmqx2', 'backend_version': '2.0.6',},
'qobj_id': '3c5712aa-4503-4a86-8266-c2c8de128a52',
'success': True,
'backend_name': 'ibmqx2',
'results': [*backend_result1.to_dict()['results'], *backend_result2.to_dict()['results']][:trunc_parameter]})
prob1 = list(map(lambda x: (x['10'] + x['11'])/8192, total_result.get_counts()))

In [ ]:
#Normal Processing of results

temp = total_result.to_dict()['results'].copy()
temp = list(map(lambda x: {'0x1': x['data']['counts']['0x1'] + x['data']['counts']['0x3'], 
                           '0x0': x['data']['counts']['0x0'] + x['data']['counts']['0x2']}, temp))
res = total_result.to_dict()['results']
real_results = total_result.to_dict().copy()
for i in range(len(res)):
    real_results['results'][i]['data']['counts'] = temp[i]
    
real_results = result.Result.from_dict(real_results)

In [ ]:
#Get rid of any results where control qubit is not 1
temp = total_result.to_dict()['results'].copy()
temp = list(map(lambda x: {'0x1': x['data']['counts']['0x3'], 
                           '0x0': x['data']['counts']['0x2']}, temp))
res = total_result.to_dict()['results']
real_results = total_result.to_dict().copy()
for i in range(len(res)):
    real_results['results'][i]['data']['counts'] = temp[i]
for i in range(len(res)):
    real_results['results'][i]['shots'] = real_results['results'][i]['data']['counts']['0x1'] + \
    real_results['results'][i]['data']['counts']['0x0']
real_results = result.Result.from_dict(real_results)

In [ ]:

plt.figure(figsize=(15, 8))
initial_theta = 0.02
initial_c = 0.5
initial_phi = 0.01


fit = AmpCalCXFitter(real_results, xdata[:trunc_parameter], qubits,
                   fit_p0=[initial_theta, initial_c],
                   fit_bounds=([-np.pi, -1],
                               [np.pi, 1]))

fit.plot(0, ax=plt.gca())
plt.scatter(xdata[:trunc_parameter], prob1)
plt.title('AmpCal for C0X1 | q0 = 1')
print("Rotation Error on C0X1: %f rads"%(fit.angle_err()[0]))
plt.show()